In [1]:
import fiber
from fiber.cohort import Cohort
from fiber.condition import Patient, MRNs
from fiber.condition import Diagnosis
from fiber.condition import Measurement, Encounter, Drug, LabValue
from fiber.storage import yaml as fiberyaml
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os
from functools import reduce 
from fiber.utils import Timer
import math


DB Password: ········


In [2]:
# Loading needed DataFrames
To_train = pq.read_table("Cohorts/ML/MEGA_FRAME/Mega_frame_with_128_drugs_and_96_lab_values_62_diseases_cleaned.parquet").to_pandas()

In [3]:
# Modifying features
To_train.drop(["age_in_years"], axis=1, inplace=True)

In [4]:
To_train

,Age_at_prediction,HT,BMI_max,BMI_mean,BMI_min,DBP_max,DBP_mean,DBP_min,SBP_max,SBP_mean,...,Diagnosis__IMO__114039,Diagnosis__IMO__134988,Diagnosis__IMO__142138,Diagnosis__IMO__15257,Diagnosis__IMO__158573,Diagnosis__IMO__158619,Diagnosis__IMO__159124,Diagnosis__IMO__209659,gender_Female,gender_Male
medical_record_number,,,,,,,,,,,,,,,,,,,,,
100067499,23923.0,1.0,25.109734,24.272838,23.674874,90.0,74.375000,53.0,149.0,133.750000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
100091035,11189.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1004105957,22781.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
100423963,23404.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1005188482,20334.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1006954100,15062.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1006987597,27438.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1007749798,27053.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1007977434,24167.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1


In [ ]:
To_train.loc[To_train['pulse_rate_min'] <15]

In [5]:
# Adjusting vital signs
To_train = To_train[(To_train.DBP_max < 150) |(To_train.DBP_max.isnull())]
To_train = To_train[(To_train.SBP_max < 250) |(To_train.SBP_max.isnull())]

To_train = To_train[(To_train.DBP_min > 20) | (To_train.DBP_min.isnull())]
To_train = To_train[(To_train.SBP_min > 50) | (To_train.SBP_min.isnull())]

To_train = To_train[(To_train.DBP_mean > 50) | (To_train.DBP_mean.isnull())]
To_train = To_train[(To_train.SBP_mean > 80) | (To_train.SBP_mean.isnull())]

To_train = To_train[To_train.Age_at_prediction > 6300]

To_train = To_train[(To_train.BMI_max < 50) | (To_train.BMI_max.isnull())]
To_train = To_train[(To_train.BMI_min > 10) | (To_train.BMI_min.isnull())]
To_train = To_train[(To_train.BMI_mean > 15) | (To_train.BMI_mean.isnull())]

To_train = To_train[(To_train.pulse_rate_mean > 50) | (To_train.pulse_rate_mean.isnull())]
To_train = To_train[(To_train.pulse_rate_max < 200) | (To_train.pulse_rate_max.isnull())]
To_train = To_train[(To_train.pulse_rate_min > 15) |  (To_train.pulse_rate_min.isnull())]


In [6]:
To_train

,Age_at_prediction,HT,BMI_max,BMI_mean,BMI_min,DBP_max,DBP_mean,DBP_min,SBP_max,SBP_mean,...,Diagnosis__IMO__114039,Diagnosis__IMO__134988,Diagnosis__IMO__142138,Diagnosis__IMO__15257,Diagnosis__IMO__158573,Diagnosis__IMO__158619,Diagnosis__IMO__159124,Diagnosis__IMO__209659,gender_Female,gender_Male
medical_record_number,,,,,,,,,,,,,,,,,,,,,
100067499,23923.0,1.0,25.109734,24.272838,23.674874,90.0,74.375000,53.0,149.0,133.750000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
100091035,11189.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1004105957,22781.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
100423963,23404.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1005188482,20334.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1006954100,15062.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1006987597,27438.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1007749798,27053.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1007977434,24167.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1


In [31]:
To_train.loc[To_train['Drug__IBEX MEDICATION__Not Taking Medication'] > 5]

,Age_at_prediction,HT,BMI_max,BMI_mean,BMI_min,DBP_max,DBP_mean,DBP_min,SBP_max,SBP_mean,...,Diagnosis__IMO__114039,Diagnosis__IMO__134988,Diagnosis__IMO__142138,Diagnosis__IMO__15257,Diagnosis__IMO__158573,Diagnosis__IMO__158619,Diagnosis__IMO__159124,Diagnosis__IMO__209659,gender_Female,gender_Male
medical_record_number,,,,,,,,,,,,,,,,,,,,,
2458090178,9641.0,1.0,26.320980,22.145781,20.659320,76.0,65.555556,60.0,137.0,112.888889,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1159968982,17430.0,1.0,37.260325,34.586928,31.937326,89.0,65.100000,51.0,143.0,120.280000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1237061973,10764.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
3942688023,11858.0,0.0,27.810213,25.917736,25.233963,74.0,64.767442,53.0,128.0,108.465116,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
735061217,10585.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,1
2644325104,16267.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1,0
3626812340,8555.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
3837667478,10125.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,0
4141403251,13747.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1,0


In [8]:
# Adjusting Lab Values
To_train = To_train[(To_train.max__LabValue__GLUCOSE < 550) | (To_train.max__LabValue__GLUCOSE.isnull())]
To_train = To_train[(To_train.median__LabValue__GLUCOSE < 350) | (To_train.median__LabValue__GLUCOSE.isnull())]
To_train = To_train[(To_train.min__LabValue__GLUCOSE > 20) | (To_train.min__LabValue__GLUCOSE.isnull())]

To_train = To_train[(To_train.min__LabValue__POTASSIUMBLD > 1) | (To_train.min__LabValue__POTASSIUMBLD.isnull())]
To_train = To_train[(To_train.median__LabValue__POTASSIUMBLD < 10) | (To_train.median__LabValue__POTASSIUMBLD.isnull())]
To_train = To_train[(To_train.max__LabValue__POTASSIUMBLD < 115) | (To_train.max__LabValue__POTASSIUMBLD.isnull())]


In [9]:
To_train

,Age_at_prediction,HT,BMI_max,BMI_mean,BMI_min,DBP_max,DBP_mean,DBP_min,SBP_max,SBP_mean,...,Diagnosis__IMO__114039,Diagnosis__IMO__134988,Diagnosis__IMO__142138,Diagnosis__IMO__15257,Diagnosis__IMO__158573,Diagnosis__IMO__158619,Diagnosis__IMO__159124,Diagnosis__IMO__209659,gender_Female,gender_Male
medical_record_number,,,,,,,,,,,,,,,,,,,,,
100067499,23923.0,1.0,25.109734,24.272838,23.674874,90.0,74.375000,53.0,149.0,133.750000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
100091035,11189.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1004105957,22781.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
100423963,23404.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1005188482,20334.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1006954100,15062.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1006987597,27438.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1007749798,27053.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1007977434,24167.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1


In [14]:
# Rename lab value columns to avoid special signs
To_Train = To_train.rename({"max__LabValue__UREA NITROGEN-BLD": "max__LabValue__UREA_NITROGEN_BLD","median__LabValue__UREA NITROGEN-BLD": "median__LabValue__UREA_NITROGEN_BLD", "min__LabValue__UREA NITROGEN-BLD": "min__LabValue__UREA_NITROGEN_BLD" }, axis =1,inplace=True)


In [18]:
To_train = To_train[(To_train.max__LabValue__UREA_NITROGEN_BLD < 80) | (To_train.max__LabValue__UREA_NITROGEN_BLD.isnull())]
To_train = To_train[(To_train.median__LabValue__UREA_NITROGEN_BLD < 50) | (To_train.median__LabValue__UREA_NITROGEN_BLD.isnull())]
To_train = To_train[(To_train.min__LabValue__UREA_NITROGEN_BLD > 2) | (To_train.min__LabValue__UREA_NITROGEN_BLD.isnull())]

In [20]:
# Rename lab value columns to avoid special signs
To_Train = To_train.rename({"min__LabValue__CREATININE-SERUM": "min__LabValue__CREATININE_SERUM",
                         "max__LabValue__CREATININE-SERUM": "max__LabValue__CREATININE_SERUM",
                         "median__LabValue__CREATININE-SERUM": "median__LabValue__CREATININE_SERUM"}, axis =1,inplace=True)

In [22]:
To_train = To_train[(To_train.min__LabValue__CREATININE_SERUM > 0.3) | (To_train.min__LabValue__CREATININE_SERUM.isnull())]
To_train = To_train[(To_train.max__LabValue__CREATININE_SERUM < 10.5) | (To_train.max__LabValue__CREATININE_SERUM.isnull())]
To_train = To_train[(To_train.median__LabValue__CREATININE_SERUM < 8.5) | (To_train.median__LabValue__CREATININE_SERUM.isnull())]

In [24]:
# Rename lab value columns to avoid special signs
To_Train = To_train.rename({"max__LabValue__MEAN CORP. HGB CONC.": "max__LabValue__MEAN_CORP_HGB_CONC",
                         "median__LabValue__MEAN CORP. HGB CONC.": "median__LabValue__MEAN_CORP_HGB_CONC",
                         "min__LabValue__MEAN CORP. HGB CONC.": "min__LabValue__MEAN_CORP_HGB_CONC"}, axis =1,inplace=True)


In [26]:
To_train = To_train[(To_train.max__LabValue__MEAN_CORP_HGB_CONC < 38) | (To_train.max__LabValue__MEAN_CORP_HGB_CONC.isnull())]
To_train = To_train[(To_train.median__LabValue__MEAN_CORP_HGB_CONC < 36) | (To_train.median__LabValue__MEAN_CORP_HGB_CONC.isnull())]
To_train = To_train[(To_train.min__LabValue__MEAN_CORP_HGB_CONC > 25) | (To_train.min__LabValue__MEAN_CORP_HGB_CONC.isnull())]


In [27]:
To_train

,Age_at_prediction,HT,BMI_max,BMI_mean,BMI_min,DBP_max,DBP_mean,DBP_min,SBP_max,SBP_mean,...,Diagnosis__IMO__114039,Diagnosis__IMO__134988,Diagnosis__IMO__142138,Diagnosis__IMO__15257,Diagnosis__IMO__158573,Diagnosis__IMO__158619,Diagnosis__IMO__159124,Diagnosis__IMO__209659,gender_Female,gender_Male
medical_record_number,,,,,,,,,,,,,,,,,,,,,
100067499,23923.0,1.0,25.109734,24.272838,23.674874,90.0,74.375000,53.0,149.0,133.750000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
100091035,11189.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1004105957,22781.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
100423963,23404.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1005188482,20334.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1006954100,15062.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1006987597,27438.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1007749798,27053.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1007977434,24167.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1


In [28]:
for col in To_train:
    print (col)

Age_at_prediction
HT
BMI_max
BMI_mean
BMI_min
DBP_max
DBP_mean
DBP_min
SBP_max
SBP_mean
SBP_min
pulse_rate_max
pulse_rate_mean
pulse_rate_min
Drug__COMPURECORD__Cefazolin
Drug__COMPURECORD__Dexamethasone
Drug__COMPURECORD__Ephedrine
Drug__COMPURECORD__Fentanyl
Drug__COMPURECORD__Glycopyrrolate
Drug__COMPURECORD__Ketorolac
Drug__COMPURECORD__Midazolam
Drug__COMPURECORD__Neostigmine
Drug__COMPURECORD__Ondansetron
Drug__COMPURECORD__Propofol
Drug__COMPURECORD__Succinylcholine
Drug__COMPURECORD__Vecuronium
Drug__EPIC IMMUNIZATION__115
Drug__EPIC IMMUNIZATION__181
Drug__EPIC IMMUNIZATION__202
Drug__EPIC IMMUNIZATION__49
Drug__EPIC IMMUNIZATION__51
Drug__EPIC IMMUNIZATION__55
Drug__EPIC IMMUNIZATION__57
Drug__EPIC IMMUNIZATION__71
Drug__EPIC IMMUNIZATION__92
Drug__EPIC MEDICATION__10039
Drug__EPIC MEDICATION__10196
Drug__EPIC MEDICATION__10279
Drug__EPIC MEDICATION__107981
Drug__EPIC MEDICATION__11000
Drug__EPIC MEDICATION__1182
Drug__EPIC MEDICATION__12134
Drug__EPIC MEDICATION__12204
Drug_

In [33]:
#To_train.to_parquet("Mega_frame_with_128_drugs_and_96_lab_values_62_diseases_cleaned_new.parquet")


In [ ]:
##Loading 70k dataframe#

In [34]:
# Loading needed DataFrames
To_train1 = pq.read_table("Cohorts/ML/Final_Model_BMI_BP_Pulse_4CV_5Drugs_wo_NaN.parquet").to_pandas()


In [35]:
To_train1

,Age_at_prediction,BMI_max,BMI_mean,BMI_min,DBP_max,DBP_mean,DBP_min,HT,SBP_max,SBP_mean,...,has_condition_ihd,has_condition_veins_and_lyphatics,number_of_occurences_Med_CV,number_of_occurences_Med_depression,number_of_occurences_Med_lipid_disorders,number_of_occurences_Med_schizo,number_of_occurences_Med_type2_diabetes,pulse_rate_max,pulse_rate_mean,pulse_rate_min
medical_record_number,,,,,,,,,,,,,,,,,,,,,
905922732,10978.0,25.836817,25.249440,24.589002,66.0,64.000000,60.0,0,110.0,100.000000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,67.0,61.500000,56.0
2579614272,18417.0,20.507829,19.506329,18.731737,76.0,70.500000,66.0,0,115.0,111.750000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,95.0,81.250000,70.0
3633798607,10878.0,34.998994,30.868551,28.803330,77.0,59.433333,49.0,0,144.0,107.451613,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,114.0,81.062500,55.0
146415158,13011.0,24.129054,24.129054,24.129054,84.0,78.250000,73.0,0,119.0,115.750000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,94.0,77.500000,60.0
4161475527,13782.0,27.817969,24.236089,19.968809,69.0,57.466667,40.0,0,126.0,105.171429,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,110.0,75.773810,54.0
24548130,9983.0,28.734672,28.080130,27.515058,74.0,72.600000,70.0,0,115.0,110.400000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78.0,71.800000,60.0
2204833684,26621.0,29.165891,29.165891,29.165891,80.0,76.666667,70.0,0,120.0,116.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0,75.000000,70.0
257369101,12118.0,27.897413,27.897413,27.897413,81.0,75.625000,67.0,0,128.0,122.250000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,98.0,90.000000,77.0
2479761144,23133.0,28.419941,27.430307,26.775716,81.0,71.000000,60.0,0,184.0,126.250000,...,0.0,0.0,0.0,4.0,1.0,0.0,0.0,85.0,74.333333,58.0


In [36]:
# Adjusting vital signs
To_train1 = To_train1[(To_train1.DBP_max < 150) |(To_train1.DBP_max.isnull())]
To_train1 = To_train1[(To_train1.SBP_max < 250) |(To_train1.SBP_max.isnull())]

To_train1 = To_train1[(To_train1.DBP_min > 20) | (To_train1.DBP_min.isnull())]
To_train1 = To_train1[(To_train1.SBP_min > 50) | (To_train1.SBP_min.isnull())]

To_train1 = To_train1[(To_train1.DBP_mean > 50) | (To_train1.DBP_mean.isnull())]
To_train1 = To_train1[(To_train1.SBP_mean > 80) | (To_train1.SBP_mean.isnull())]

To_train1 = To_train1[To_train1.Age_at_prediction > 6300]

To_train1 = To_train1[(To_train1.BMI_max < 50) | (To_train1.BMI_max.isnull())]
To_train1 = To_train1[(To_train1.BMI_min > 10) | (To_train1.BMI_min.isnull())]
To_train1 = To_train1[(To_train1.BMI_mean > 15) | (To_train1.BMI_mean.isnull())]

To_train1 = To_train1[(To_train1.pulse_rate_mean > 50) | (To_train1.pulse_rate_mean.isnull())]
To_train1 = To_train1[(To_train1.pulse_rate_max < 200) | (To_train1.pulse_rate_max.isnull())]
To_train1 = To_train1[(To_train1.pulse_rate_min > 15) |  (To_train1.pulse_rate_min.isnull())]


In [37]:
To_train1

,Age_at_prediction,BMI_max,BMI_mean,BMI_min,DBP_max,DBP_mean,DBP_min,HT,SBP_max,SBP_mean,...,has_condition_ihd,has_condition_veins_and_lyphatics,number_of_occurences_Med_CV,number_of_occurences_Med_depression,number_of_occurences_Med_lipid_disorders,number_of_occurences_Med_schizo,number_of_occurences_Med_type2_diabetes,pulse_rate_max,pulse_rate_mean,pulse_rate_min
medical_record_number,,,,,,,,,,,,,,,,,,,,,
905922732,10978.0,25.836817,25.249440,24.589002,66.0,64.000000,60.0,0,110.0,100.000000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,67.0,61.500000,56.0
2579614272,18417.0,20.507829,19.506329,18.731737,76.0,70.500000,66.0,0,115.0,111.750000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,95.0,81.250000,70.0
3633798607,10878.0,34.998994,30.868551,28.803330,77.0,59.433333,49.0,0,144.0,107.451613,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,114.0,81.062500,55.0
146415158,13011.0,24.129054,24.129054,24.129054,84.0,78.250000,73.0,0,119.0,115.750000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,94.0,77.500000,60.0
4161475527,13782.0,27.817969,24.236089,19.968809,69.0,57.466667,40.0,0,126.0,105.171429,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,110.0,75.773810,54.0
24548130,9983.0,28.734672,28.080130,27.515058,74.0,72.600000,70.0,0,115.0,110.400000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78.0,71.800000,60.0
2204833684,26621.0,29.165891,29.165891,29.165891,80.0,76.666667,70.0,0,120.0,116.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0,75.000000,70.0
257369101,12118.0,27.897413,27.897413,27.897413,81.0,75.625000,67.0,0,128.0,122.250000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,98.0,90.000000,77.0
2479761144,23133.0,28.419941,27.430307,26.775716,81.0,71.000000,60.0,0,184.0,126.250000,...,0.0,0.0,0.0,4.0,1.0,0.0,0.0,85.0,74.333333,58.0


In [38]:
#To_train1.to_parquet("Cleaned_Final_Model_BMI_BP_Pulse_4CV_5Drugs_wo_NaN.parquet")

In [39]:
# Loading 300k imputed model 
To_train2 = pq.read_table("Cohorts/ML/Final_Model_BMI_pulse_imputed_300k.parquet").to_pandas()


In [42]:
To_train2

,Age_at_prediction,HT,gender_Female,gender_Male,has_condition_ihd,number_of_occurences_ihd,has_condition_cr,number_of_occurences_cr,has_condition_veins_and_lyphatics,number_of_occurences_veins_and_lyphatics,...,has_condition_Med_schizo,number_of_occurences_Med_schizo,has_condition_Med_type2_diabetes,number_of_occurences_Med_type2_diabetes,BMI_max,BMI_mean,BMI_min,pulse_rate_max,pulse_rate_mean,pulse_rate_min
medical_record_number,,,,,,,,,,,,,,,,,,,,,
100067499,23923.0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.109734,24.272838,23.674874,108.0,85.105691,62.0
100091035,11189.0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,35.441024,32.929437,30.746955,124.0,74.607692,59.0
1004105957,22781.0,1,0,1,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,30.561949,29.959300,28.782925,92.0,72.872727,57.0
100423963,23404.0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,27.897413,27.823544,27.749675,90.0,86.000000,82.0
1005188482,20334.0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,38.637439,35.981394,33.665377,110.0,82.392157,63.0
1006954100,15062.0,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,43.745619,40.304674,35.719821,97.0,81.333333,70.0
1006987597,27438.0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,39.865334,38.913504,38.273520,100.0,79.412500,58.0
1007749798,27053.0,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,22.793513,21.715046,20.800858,82.0,70.586207,60.0
1007977434,24167.0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,30.119890,28.311455,26.820759,76.0,64.750000,50.0


In [43]:
# Adjusting vital signs
To_train2 = To_train2[To_train2.Age_at_prediction > 6300]

To_train2 = To_train2[(To_train2.BMI_max < 50) | (To_train2.BMI_max.isnull())]
To_train2 = To_train2[(To_train2.BMI_min > 10) | (To_train2.BMI_min.isnull())]
To_train2 = To_train2[(To_train2.BMI_mean > 15) | (To_train2.BMI_mean.isnull())]

To_train2 = To_train2[(To_train2.pulse_rate_mean > 50) | (To_train2.pulse_rate_mean.isnull())]
To_train2 = To_train2[(To_train2.pulse_rate_max < 200) | (To_train2.pulse_rate_max.isnull())]
To_train2 = To_train2[(To_train2.pulse_rate_min > 15) |  (To_train2.pulse_rate_min.isnull())]


In [44]:
To_train2

,Age_at_prediction,HT,gender_Female,gender_Male,has_condition_ihd,number_of_occurences_ihd,has_condition_cr,number_of_occurences_cr,has_condition_veins_and_lyphatics,number_of_occurences_veins_and_lyphatics,...,has_condition_Med_schizo,number_of_occurences_Med_schizo,has_condition_Med_type2_diabetes,number_of_occurences_Med_type2_diabetes,BMI_max,BMI_mean,BMI_min,pulse_rate_max,pulse_rate_mean,pulse_rate_min
medical_record_number,,,,,,,,,,,,,,,,,,,,,
100067499,23923.0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.109734,24.272838,23.674874,108.0,85.105691,62.0
100091035,11189.0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,35.441024,32.929437,30.746955,124.0,74.607692,59.0
1004105957,22781.0,1,0,1,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,30.561949,29.959300,28.782925,92.0,72.872727,57.0
100423963,23404.0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,27.897413,27.823544,27.749675,90.0,86.000000,82.0
1005188482,20334.0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,38.637439,35.981394,33.665377,110.0,82.392157,63.0
1006954100,15062.0,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,43.745619,40.304674,35.719821,97.0,81.333333,70.0
1006987597,27438.0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,39.865334,38.913504,38.273520,100.0,79.412500,58.0
1007749798,27053.0,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,22.793513,21.715046,20.800858,82.0,70.586207,60.0
1007977434,24167.0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,30.119890,28.311455,26.820759,76.0,64.750000,50.0


In [45]:
#To_train2.to_parquet("Cleaned_Final_Model_BMI_pulse_imputed_300k.parquet")